## Part 3. Recommendation

For the baseline, our model selects the top 10 predicted ratings and their respective items. 

In [1]:
import time
import pickle

import numpy as np 
import torch

import utils as UT 

#np.random.seed(40) # fixed random seed

In [2]:
# load trained model
model = torch.load("checkpoint.pt")
with open("user_ratings.pickle", "rb") as f:
    user_ratings = pickle.load(f)[0]

In [3]:
# prepare training data
m, n = user_ratings.shape
k = 10
#bitmask = np.random.rand(m, n) > 0.8
bitmask = UT.generate_bitmask(user_ratings)
train = user_ratings.copy()
train[bitmask] = 0
test = user_ratings - train
test_size = np.count_nonzero(test)

In [4]:
# vectorized test 
model.eval()

user_embeddings = model.U.weight.transpose(0,1).unsqueeze(1)
item_embeddings = model.V.weight.transpose(0,1).unsqueeze(2)
product = torch.matmul(item_embeddings, user_embeddings).transpose(0,2)
pred_torch = model.A_pred(product).squeeze()

optional: to assure the correctness of matrix multiplications
``` python
# backup: per element test
predictions = np.zeros((m, n))
test_idx = np.nonzero(user_ratings - train)
X, Y = test_idx

print("total test data:",len(X))

for i in range(len(X)):
    x, y = X[i], Y[i]
    xl = torch.LongTensor([x])
    yl = torch.LongTensor([y])

    pred = model(xl, yl).item()
    #pred = pred / 4 * 5 # HACKS!!!!
    predictions[x, y] = pred

#print(torch.sum(pred_torch - torch.from_numpy(predictions))) # difference between vectorized and element-wise tests
```

In [5]:
# compute basic errors using the test set
predictions = pred_torch.detach().numpy()
pred_copy = predictions.copy()
pred_copy[test==0] = 0 # delete those cannot be tested (not in test data)

rmse = np.sqrt(np.sum(np.square(test-pred_copy)) / test_size) # rmse
mae = np.sum(test-pred_copy) / test_size # mae

print("rmse: %.2f"%rmse)
print("mae: %.2f"%mae)

rmse: 1.48
mae: 0.85


In [7]:
# avoids training data
predictions[train!=0] = 0

# compute different scores
# methodology=0: using also predictions on user-item pairs beyond this dataset
# methodology=1: using only predictions that have their corresponding ground truths
prec, rec, F, cr, ndcg = UT.compute_scores(prediction=predictions.copy(), 
                                            ground_truth=test.copy(), 
                                            K=10,
                                            w=-2,
                                            methodology=1)
print("Precision: %.2f %%\nRecall: %.2f %%\nF: %.2f\nConversion Rate: %.2f %%\nNDCG: %.2f"%(prec*100, rec*100, F, cr*100, ndcg))

Precision: 95.05 %
Recall: 93.04 %
F: 0.94
Conversion Rate: 99.81 %
NDCG: 0.95
